In [0]:
''''
Author : Viraj Kamat
Objective: Identify couples from florida voter registration data


'''


import pandas as pd
import pdb
import os
from google.colab import drive

from datetime import datetime
drive.mount('/content/drive')


#Read all files and merge into 1 ( All counties)

dataframes = []

counter = 0
total = len(os.listdir('/content/drive/My Drive/20160930')) 
for filename in os.listdir('/content/drive/My Drive/20160930'):
    if counter%10 == 0 :
      print("Processed {} of {} files".format(counter,total))
      print('\n')
    
    counter += 1
    if filename.endswith(".txt") or filename.endswith(".py"): 
        data = pd.read_csv('/content/drive/My Drive/20160930/'+filename, sep="\t", header=None,low_memory=False)
        
        data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]

        data =  data.loc[:,["Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]
        
        data = data.dropna(subset=["Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])
        
        dataframes.append(data)

print("Processed {} of {} files".format(counter,total))
print('\n')

new_dataframe = pd.DataFrame()
temp_dataframe = pd.DataFrame()

#for i,df in enumerate(dataframes) :
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
#print(temp_dataframe)

#pdb.set_trace()

#Add columns




#Filter out irrelevant columns


#Identify couples


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processed 0 of 70 files


Processed 10 of 70 files




In [0]:
#print (temp_dataframe.head(5))
temp=temp_dataframe.sample(n=20000)
#print(temp.head(5))
#print(temp['Residence Address Line 2'].head(5))
temp['All'] = temp['Residence Address Line 1'].str.cat(temp[['Residence Address Line 2']], sep=' - ')
#print(temp['All'].head(5))
df=temp.groupby('All')
data=pd.DataFrame()
for name,group in df:
    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                print(name1)
                print(group1)
                data=data.append([group])
#print(data)

        

In [0]:

pd.options.display.max_columns = None
temp2 = temp_dataframe
temp2 = temp2.sort_values(by=['Name Last'])
temp2 = temp2.iloc[0:1000000,:] 
temp2['All'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')
df=temp2.groupby('All')
data=pd.DataFrame()

df1 = pd.DataFrame()
df2 = pd.DataFrame()

counter = 0
print(temp2.shape)
for name,group in df:

    if counter%30000 == 0 :
      print("Processed {} {} of items".format(counter,temp2.shape[0]) )
    counter += 1

    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                #print(name1)
                #print(group1)
                data=data.append([group])
                #pdb.set_trace()
                if (group1.iloc[0,12] == group1.iloc[1,12]) :
                  df1 = df1.append(group)
                else :
                  df2 = df2.append(group)

print("Processed {} {} of items".format(counter,temp2.shape[0]) )
print('Processing Complete')



(1000000, 14)
Processed 0 1000000 of items
Processed 30000 1000000 of items
Processed 60000 1000000 of items
Processed 90000 1000000 of items
Processed 120000 1000000 of items
Processed 150000 1000000 of items
Processed 180000 1000000 of items
Processed 210000 1000000 of items
Processed 240000 1000000 of items
Processed 270000 1000000 of items
Processed 300000 1000000 of items
Processed 330000 1000000 of items
Processed 360000 1000000 of items
Processed 390000 1000000 of items
Processed 420000 1000000 of items
Processed 450000 1000000 of items
Processed 480000 1000000 of items
Processed 510000 1000000 of items
Processed 540000 1000000 of items
Processed 570000 1000000 of items


In [0]:
df5= temp_dataframe.groupby('Gender') 

for name,group3 in df5:
    if name=='F':
      grouping=group3.groupby('Party Affiliation')
      for name5,group5 in grouping:
        if name5 == 'DEM':
          demF=group5.shape[0]
        elif name5 == 'REP':
            repF=group5.shape[0]
    if name=='M':
      grouping=group3.groupby('Party Affiliation')
      for name5,group5 in grouping:
        if name5 == 'DEM':
          demM=group5.shape[0]
        elif name5 == 'REP':
            repM=group5.shape[0]
List=[demF,repF,demM,repM]
Data=['DEM Female','REP Female','DEM Male','REP Male']
data2={'Name':Data,'count':List}
df_fin=pd.DataFrame(data2)
print(df_fin.head(5))
colormap=['green','DarkBlue','green','DarkBlue']
plot1=df_fin.plot(kind='bar',x="Name",y="count",title='Party',color=colormap,figsize=(8,4))

In [0]:
''''
Author : Rutvik Parekh
Objective: Finding divorce rates for couples with same and different political party affiliations 

'''


#########################################  Take the data from the 20160930 election month and store it in a dataframe. #################################

import pandas as pd
import pdb
import os
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive')

#Read all files and merge into 1 ( All counties)

dataframes = []

counter = 0
total = len(os.listdir('/content/drive/My Drive/dsf-project/3Counties_2016/')) 
for filename in os.listdir('/content/drive/My Drive/dsf-project/3Counties_2016/'):
  data = pd.read_csv('/content/drive/My Drive/dsf-project/3Counties_2016/'+filename, sep="\t", header=None,low_memory=False)        
  data.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data =  data.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]        
  data = data.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])          
  dataframes.append(data)

print("Processed {} of {} files".format(counter,total))
print('\n')

new_dataframe = pd.DataFrame()
temp_dataframe = pd.DataFrame()

#for i,df in enumerate(dataframes) :
temp_dataframe = pd.concat(dataframes)
temp_dataframe = temp_dataframe[temp_dataframe['Party Affiliation'].isin(['DEM', 'REP'])]
print(temp_dataframe.shape)



######################### Process the 20160930 voter registration data and filter the couples based on same and different political party affiliations ##################

# Separate the couples who have same and different party affiliations from 20160930 election data.

#print (temp_dataframe.head(5))
pd.options.display.max_columns = None
#temp=temp_dataframe.sample(n=20000)
temp2 = temp_dataframe
#temp2=temp_dataframe
#print(temp.head(5))
#print(temp['Residence Address Line 2'].head(5))
temp2['All'] = temp2['Residence Address Line 1'].str.cat(temp2[['Residence Address Line 2']], sep=' - ')
#print(temp['All'].head(5))
df=temp2.groupby('All')
data=pd.DataFrame()

df1 = pd.DataFrame()
df2 = pd.DataFrame()
couples1 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Voter_ID_Female", "Name_First_Female", "Name_Last_Female"])
couples2 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male", "Voter_ID_Female", "Name_First_Female", "Name_Last_Female"])
counter = 0
print(temp2.shape)
for name,group in df:

    if counter%30000 == 0 :
      print("Processed {} of items".format(counter) )
    counter += 1

    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                #print(name1)
                #print(group1)
                data=data.append([group])
                birthdate1 = pd.to_datetime(group1.iloc[0]['Birth Date'])
                birthdate2 = pd.to_datetime(group1.iloc[1]['Birth Date'])
                birthdateDiff = abs(birthdate1 - birthdate2)
                #birthdateDiffInt = birthdateDiff.astype(int)
                fifteenYears = pd.Timedelta(15, 'Y')
                if birthdateDiff < fifteenYears:
                  #pdb.set_trace()
                  if (group1.iloc[0,13] == group1.iloc[1,13]) :
                    df1 = df1.append(group)
                    couples1AllColumnsMale = pd.DataFrame()
                    couples1AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples1 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples1AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples1AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples1[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples1AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples1['Voter_ID_Female'] = couples1AllColumnsFemale.to_numpy()[0][0]
                    tempCouples1['Name_First_Female'] = couples1AllColumnsFemale.to_numpy()[0][2]
                    tempCouples1[ 'Name_Last_Female'] = couples1AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples1 = couples1.append(tempCouples1)
                  else :
                    df2 = df2.append(group)
                    couples2AllColumnsMale = pd.DataFrame()
                    couples2AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples2 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples2AllColumnsMale = group1[group1.Gender == 'M']
                    couples2AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples2[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples2AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples2['Voter_ID_Female'] = couples2AllColumnsFemale.to_numpy()[0][0]
                    tempCouples2['Name_First_Female'] = couples2AllColumnsFemale.to_numpy()[0][2]
                    tempCouples2[ 'Name_Last_Female'] = couples2AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples2 = couples2.append(tempCouples2)
print('Processing Complete')




########################################################## Take the data from the 20170630 election month and store it in a dataframe. ###############################
#drive.mount('/content/drive')


#Read all files and merge into 1 ( All counties)

dataframes_2017 = []
total_2017 = len(os.listdir('/content/drive/My Drive/dsf-project/3Counties_2017/')) 
for filename in os.listdir('/content/drive/My Drive/dsf-project/3Counties_2017/'):
  data_2017 = pd.read_csv('/content/drive/My Drive/dsf-project/3Counties_2017/'+filename, sep="\t", header=None,low_memory=False)       
  data_2017.columns = ["County Code","Voter ID","Name Last","Name Suffix","Name First","Name Middle","Requested public records exemption","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Mailing Address Line 1","Mailing Address Line 2","Mailing Address Line 3","Mailing City","Mailing State","Mailing Zipcode	","Mailing Country","Gender","Race","Birth Date","Registration Date","Party Affiliation","Precinct","Precinct Group","Precinct Split","Precinct Suffix","Voter Status","Congressional District","House District","Senate District","County Commission District","School Board District","Daytime Area Code","Daytime Phone Number","Daytime Phone Extension","Email address"]
  data_2017 =  data_2017.loc[:,["Voter ID","Name Last","Name First","Name Middle","Residence Address Line 1","Residence Address Line 2","Residence City (USPS)","Residence State","Residence Zipcode","Gender","Race","Birth Date","Registration Date","Party Affiliation"]]
  data_2017 = data_2017.dropna(subset=["Voter ID", "Name Last","Birth Date","Name First","Residence Address Line 1","Residence Address Line 2","Gender","Party Affiliation"])        
  dataframes_2017.append(data_2017)
print('\n')
temp_dataframe_2017 = pd.DataFrame()

#for i,df in enumerate(dataframes) :
temp_dataframe_2017 = pd.concat(dataframes_2017)
temp_dataframe_2017 = temp_dataframe_2017[temp_dataframe_2017['Party Affiliation'].isin(['DEM', 'REP'])]
print(temp_dataframe_2017.shape)




######################### Process the 201760630 voter registration data and filter the couples based on same and different political party affiliations. ##############################

#print (temp_dataframe.head(5))
pd.options.display.max_columns = None
#temp=temp_dataframe.sample(n=20000)
temp2_2017 = temp_dataframe_2017
#temp2=temp_dataframe
#print(temp.head(5))
#print(temp['Residence Address Line 2'].head(5))
temp2_2017['All'] = temp2_2017['Residence Address Line 1'].str.cat(temp2_2017[['Residence Address Line 2']], sep=' - ')
#print(temp['All'].head(5))
df_2017=temp2_2017.groupby('All')
data_2017=pd.DataFrame()

df1_2017 = pd.DataFrame()
df2_2017 = pd.DataFrame()
couples12017 = pd.DataFrame(columns=['Voter_ID_Male', 'Name_First_Male', 'Name_Last_Male', 'Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female'])
couples22017 = pd.DataFrame(columns=['Voter_ID_Male', 'Name_First_Male', 'Name_Last_Male', 'Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female'])
counter = 0
print(temp2_2017.shape)
for name,group in df_2017:

    if counter%30000 == 0 :
      print("Processed {} of items".format(counter) )
    counter += 1

    if group.shape[0]==2 and group[group.Gender == 'F'].shape[0] == 1 and group[group.Gender == 'M'].shape[0] == 1:
        group_name=group.groupby('Name Last')
        for name1,group1 in group_name:
            if group1.shape[0]==2:
                data_2017=data_2017.append([group])
                birthdate1 = pd.to_datetime(group1.iloc[0]['Birth Date'])
                birthdate2 = pd.to_datetime(group1.iloc[1]['Birth Date'])
                birthdateDiff = abs(birthdate1 - birthdate2)
                #birthdateDiffInt = birthdateDiff.astype(int)
                fifteenYears = pd.Timedelta(15, 'Y')
                if birthdateDiff < fifteenYears:
                  if (group1.iloc[0,13] == group1.iloc[1,13]) :
                    df1_2017 = df1_2017.append(group)
                    couples12017AllColumnsMale = pd.DataFrame()
                    couples12017AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples12017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples12017AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples12017AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples12017[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples12017AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples12017['Voter_ID_Female'] = couples12017AllColumnsFemale.to_numpy()[0][0]
                    tempCouples12017['Name_First_Female'] = couples12017AllColumnsFemale.to_numpy()[0][2]
                    tempCouples12017[ 'Name_Last_Female'] = couples12017AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples12017 = couples12017.append(tempCouples12017)
                  else :
                    df2_2017 = df2_2017.append(group)
                    couples22017AllColumnsMale = pd.DataFrame()
                    couples22017AllColumnsFemale = pd.DataFrame()
                    #display(group1.head())
                    #display(group1.head())
                    tempCouples22017 = pd.DataFrame(columns=["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"])
                    couples22017AllColumnsMale = group1[group1.Gender == 'M']
                    #display(group1.take([0]))
                    #display(couples1AllColumnsFemale.head())
                    couples22017AllColumnsFemale = group1[group1.Gender == 'F']
                    tempCouples22017[["Voter_ID_Male", "Name_First_Male", "Name_Last_Male"]] = couples22017AllColumnsMale[['Voter ID', 'Name First', 'Name Last']]
                    #tempCouples1[['Voter_ID_Female', 'Name_First_Female', 'Name_Last_Female']] = couples1AllColumnsFemale[['Voter ID', 'Name First', 'Name Last']]
                    tempCouples22017['Voter_ID_Female'] = couples22017AllColumnsFemale.to_numpy()[0][0]
                    tempCouples22017['Name_First_Female'] = couples22017AllColumnsFemale.to_numpy()[0][2]
                    tempCouples22017[ 'Name_Last_Female'] = couples22017AllColumnsFemale.to_numpy()[0][1]
                    #display(tempCouples1.head())
                    couples22017 = couples22017.append(tempCouples22017)



############################################# Check whether we are getting proper number of couples for all categories. ######################

display(couples1.shape)
display(couples2.shape)
display(couples12017.shape)
display(couples22017.shape)





########################################## Code for finding the divorce rate for same and different political party affiliation couples. ########################

import numpy as np
couples_samePPA_merged = pd.merge(couples1, couples12017, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
#couples_samePPA_merged.drop(['Name_First_Male','Name_Last_Male','Name_First_Female','Name_Last_Female'], inplace=True, axis=1)
display(couples_samePPA_merged.head())
couples_samePPA_merged[['Exist']] = np.where(couples_samePPA_merged.Exist == 'both', True, False)
display(couples_samePPA_merged['Exist'].value_counts())

samePPA_valueCounts = couples_samePPA_merged['Exist'].value_counts().tolist()
sameDivorceRate = samePPA_valueCounts[1]/(samePPA_valueCounts[0]+samePPA_valueCounts[1])
couples_diffPPA_merged = pd.merge(couples2, couples22017, on=['Voter_ID_Male','Voter_ID_Female'], how='left', indicator='Exist')
#couples_diffPPA_merged.drop(['Name_First_Male','Name_Last_Male','Name_First_Female','Name_Last_Female'], inplace=True, axis=1)
couples_diffPPA_merged['Exist'] = np.where(couples_diffPPA_merged.Exist == 'both', True, False)

display(couples_diffPPA_merged['Exist'].value_counts())

diffPPA_valueCounts = couples_diffPPA_merged['Exist'].value_counts().tolist()
diffDivorceRate = diffPPA_valueCounts[1]/(diffPPA_valueCounts[0]+diffPPA_valueCounts[1])


##################################### Print the divorce rates for same and different political party affiliation couples. #######################

print("Same political party affiliation divorce rate:")
display(sameDivorceRate*100)
print("Different political party affiliation divorce rate:")
display(diffDivorceRate*100)

In [0]:
print(couples_samePPA_merged['Exist'].value_counts().tolist())
